In [1]:
import pandas as pd
import sqlite3
from datetime import datetime

# to simulate chunk loading
from tqdm import tqdm


In [2]:
from google.colab import drive
drive.mount('/content/drive')

# Update the path to your cleaned file
csv_path = '/content/drive/MyDrive/capstone-data/cleaned_combined_data_2007.csv'

# Load the cleaned data
df = pd.read_csv(csv_path, parse_dates=['datetime'])
df.set_index('datetime', inplace=True)
df.columns = df.columns.str.lower()

df.head()



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,global_active_power,global_reactive_power,voltage,global_intensity,sub_metering_1,sub_metering_2,sub_metering_3,temp,humidity,windspeed,cloudcover
datetime,,,,,,,,,,,
2007-01-01 00:00:00,2.550633,0.112800,241.361333,10.530000,0.0,0.583333,0.0,13.9,86.47,30.9,95.5
2007-01-01 01:00:00,2.523400,0.071633,241.096500,10.443333,0.0,0.000000,0.0,14.0,85.10,25.5,96.9
2007-01-01 02:00:00,2.582333,0.106667,243.200167,10.540000,0.0,0.333333,0.0,14.0,84.29,27.7,92.7
2007-01-01 03:00:00,2.541667,0.090100,243.265667,10.400000,0.0,0.266667,0.0,13.8,80.82,21.7,67.7
2007-01-01 04:00:00,2.475733,0.088167,242.456167,10.110000,0.0,0.000000,0.0,12.9,59.76,26.8,23.7


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 7357 entries, 2007-01-01 00:00:00 to 2007-12-31 23:00:00
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   global_active_power    7277 non-null   float64
 1   global_reactive_power  7277 non-null   float64
 2   voltage                7277 non-null   float64
 3   global_intensity       7277 non-null   float64
 4   sub_metering_1         7277 non-null   float64
 5   sub_metering_2         7277 non-null   float64
 6   sub_metering_3         7277 non-null   float64
 7   temp                   7357 non-null   float64
 8   humidity               7357 non-null   float64
 9   windspeed              7357 non-null   float64
 10  cloudcover             7357 non-null   float64
dtypes: float64(11)
memory usage: 689.7 KB


In [4]:
# Create or connect to a local SQLite database
conn = sqlite3.connect("energy_timeseries.db")

# Define table schema using SQL
conn.execute('''
CREATE TABLE IF NOT EXISTS timeseries_energy_weather (
    datetime TEXT PRIMARY KEY,
    global_active_power REAL,
    voltage REAL,
    sub_metering_1 REAL,
    sub_metering_2 REAL,
    sub_metering_3 REAL,
    temp REAL,
    humidity REAL,
    windspeed REAL,
    cloudcover REAL
);
''')

conn.commit()


In [5]:
def insert_daily_chunk(data, date_str, conn):
    # Filter data for the given date
    day_data = data[data.index.date == pd.to_datetime(date_str).date()]

    if not day_data.empty:
        columns_needed = [
            'global_active_power', 'voltage',
            'sub_metering_1', 'sub_metering_2', 'sub_metering_3',
            'temp', 'humidity', 'windspeed', 'cloudcover'
        ]
        filtered_data = day_data[columns_needed].copy()

        # Add datetime column from index in exact string format
        filtered_data['datetime'] = day_data.index.strftime('%Y-%m-%d %H:%M:%S')

        # Query all existing datetimes for that day
        day_start = date_str + " 00:00:00"
        day_end = date_str + " 23:59:59"

        query = f"""
            SELECT datetime
            FROM timeseries_energy_weather
            WHERE datetime BETWEEN '{day_start}' AND '{day_end}'
        """
        existing = pd.read_sql_query(query, conn)
        existing_dates = set(existing['datetime'])

        # Filter out already existing datetimes
        filtered_data = filtered_data[~filtered_data['datetime'].isin(existing_dates)]

        # Insert one row at a time to avoid constraint errors
        if not filtered_data.empty:
            for _, row in filtered_data.iterrows():
                try:
                    row_df = pd.DataFrame([row])
                    row_df.to_sql('timeseries_energy_weather', conn, if_exists='append', index=False)
                except:
                    pass


In [6]:
for day in pd.date_range(start='2007-01-01', end='2007-12-31'):
    date_str = day.strftime('%Y-%m-%d')
    insert_daily_chunk(df, date_str, conn)

In [7]:
# Run a sample query
sample = pd.read_sql_query(
    "SELECT * FROM timeseries_energy_weather ORDER BY datetime LIMIT 5;", conn)
sample


,datetime,global_active_power,voltage,sub_metering_1,sub_metering_2,sub_metering_3,temp,humidity,windspeed,cloudcover
0,2007-01-01 00:00:00,2.550633,241.361333,0.0,0.583333,0.0,13.9,86.47,30.9,95.5
1,2007-01-01 01:00:00,2.523400,241.096500,0.0,0.000000,0.0,14.0,85.10,25.5,96.9
2,2007-01-01 02:00:00,2.582333,243.200167,0.0,0.333333,0.0,14.0,84.29,27.7,92.7
3,2007-01-01 03:00:00,2.541667,243.265667,0.0,0.266667,0.0,13.8,80.82,21.7,67.7
4,2007-01-01 04:00:00,2.475733,242.456167,0.0,0.000000,0.0,12.9,59.76,26.8,23.7


In [8]:
# Run a sample query
sample = pd.read_sql_query(
    "SELECT count(*) FROM timeseries_energy_weather;", conn)
sample

,count(*)
0,7356


In [9]:
# Save the SQLite DB to Google Drive
!cp energy_timeseries.db "/content/drive/MyDrive/capstone-data/energy_timeseries.db"


In [10]:
df.columns


Index(['global_active_power', 'global_reactive_power', 'voltage',
       'global_intensity', 'sub_metering_1', 'sub_metering_2',
       'sub_metering_3', 'temp', 'humidity', 'windspeed', 'cloudcover'],
      dtype='object')

In [11]:
pd.read_sql_query("SELECT COUNT(*) AS total_rows FROM timeseries_energy_weather;", conn)

,total_rows
0,7356
